In [2]:
!pip install open-interpreter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 11.5 MB/s eta 0:00:0031m40.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [34]:
from interpreter import OpenInterpreter
import openai

In [2]:
def print_agent_attrs(agent):
    # Get the dictionary of attributes
    attributes = agent.__dict__
    
    # Create a list of tuples containing key and type of value
    attributes_with_types = [(key, type(value).__name__) for key, value in attributes.items()]
    
    # Print the result
    for key, value_type in attributes_with_types:
        print(f"{key}: {value_type}")

In [41]:
def util_read_file(filename):
    with open(filename, 'r') as f:
        content = f.read()
    return content

def get_prompt(name):
    filepath = f"prompts/{name}.txt"
    return util_read_file(filepath)

def get_current_task():
    return util_read_file("current_task.txt")

In [4]:
import time

class FIFOQueue:
    def __init__(self):
        self.queue = []

    def enqueue(self, items):
        timestamp = time.time()
        if not isinstance(items, list):
            items = [items]
        self.queue.append({"timestamp": timestamp, "messages": items})

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        else:
            raise IndexError("Dequeue from an empty queue")

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)


In [51]:
import openai
import os

class Agent:
    def __init__(self, name, special_commands="", mode="OpenInterpreter", context_window=128000, max_tokens=10000):
        self.name = name
        self.states = ["SPECTATE", "MUTATE", "DICTATE"]
        self.state = "SPECTATE"
        self.inbox = FIFOQueue()
        self.internal_state = []
        self.mode = mode
        self.model = "gpt-4o-mini"
        self.context_window = context_window
        self.max_tokens = max_tokens
        self.generated_new_messages = []
        self.most_recently_received_messages = []
        self.other_agents = []

        if self.mode == "OpenInterpreter":
            self.oi = OpenInterpreter()
            self.oi.llm.model = self.model
            self.oi.llm.context_window = self.context_window
            self.oi.llm.max_tokens = self.max_tokens
            self.oi.system_message += special_commands
            self.system_message = self.oi.system_message
            self.oi.auto_run = True
            self.oi.loop = True
        elif self.mode == "OpenAI":
            self.system_message = special_commands

    def add_other_agent(self, agent):
        self.other_agents.append(agent)

    def handle_state(self):
        print(f"[{self.name}] Current State: {self.state}")

        if self.state == "SPECTATE":
            if not self.inbox.is_empty():
                self.most_recently_received_messages = self.inbox.dequeue()["messages"]
                print(f"[{self.name}] Received message(s)")
                self.state = "MUTATE"

        elif self.state == "MUTATE":
            self.internal_state.extend(self.most_recently_received_messages)
            print(f"[{self.name}] Internal state mutated")
            messages = [{"role": "user", "content": msg["content"], "type": "message"} for msg in self.most_recently_received_messages]
            if self.mode == "OpenInterpreter":
                self.generated_new_messages = self.oi.chat(messages, display=True, stream=False, blocking=True)
            elif self.mode == "OpenAI":
                self.generated_new_messages = self.openai_chat(messages)
            print(f"[{self.name}] Generated new message(s)")
            if self.mode == "OpenAI":
                print(f"[{self.name}] New message(s): {self.generated_new_messages}")
            self.state = "DICTATE"

        elif self.state == "DICTATE":
            self.send_message_to_agents(self.generated_new_messages)
            self.generated_new_messages = []
            self.most_recently_received_messages = []
            self.state = "SPECTATE"

        print(f"[{self.name}] New State: {self.state}")

    def mutate_state(self, internal_state):
        internal_state.append({'mutated': True})
        print(f"[{self.name}] Internal state mutated")
        return internal_state

    def send_message_to_agents(self, messages):
        for agent in self.other_agents:
            agent.inbox.enqueue(messages)
            print(f"[{self.name}] Sent message to {agent.name}")

    def swap_roles(self, messages):
        for message in messages:
            if message['role'] == 'user':
                message['role'] = 'assistant'
            elif message['role'] == 'assistant':
                message['role'] = 'user'
        return messages

    def openai_chat(self, messages):
        if self.mode != "OpenAI":
            return None
            
        response_content = OpenAI().chat.completions.create(
            model=self.model,
            max_tokens=self.max_tokens,
            messages=[
                {
                    "role":"system",
                    "content": self.system_message
                },
                *messages
            ],
        ).choices[0].message.content
        return [{"role":"assistant", "content": response_content, "type": "message"}]



In [ ]:

roadmapper = Agent("Roadmap Architect", get_prompt("roadmapper"))
pm = Agent("Project Manager", get_prompt("manager"))
dev = Agent("Software Developer", get_prompt("dev"))

roadmapper.add_other_agent(pm)
pm.add_other_agent(dev)
dev.add_other_agent(pm)

roadmapper.inbox.enqueue(
    [{"role": "user", "content": f"Here is the content of the email_exchange.txt file: \n\n {util_read_file("email_exchange.txt")}", "type": "message"},
    {"role": "user", "content": f"Here is the content of the project_description.txt file: \n\n {util_read_file("project_description.txt")}", "type": "message"},
    {"role": "user", "content": "Please proceed to update the roadmap.json file", "type": "message"}])

# Run the state handler function to control the agents in a loop
for _ in range(100):
    roadmapper.handle_state()
    pm.handle_state()
    dev.handle_state()

[Roadmap Architect] Current State: SPECTATE
[Roadmap Architect] Received message(s)
[Roadmap Architect] New State: MUTATE
[Project Manager] Current State: SPECTATE
[Project Manager] New State: SPECTATE
[Software Developer] Current State: SPECTATE
[Software Developer] New State: SPECTATE
[Roadmap Architect] Current State: MUTATE
[Roadmap Architect] Internal state mutated


Output()

Output()

Output()

[Roadmap Architect] Generated new message(s)
[Roadmap Architect] New State: DICTATE
[Project Manager] Current State: SPECTATE
[Project Manager] New State: SPECTATE
[Software Developer] Current State: SPECTATE
[Software Developer] New State: SPECTATE
[Roadmap Architect] Current State: DICTATE
[Roadmap Architect] Sent message to Project Manager
[Roadmap Architect] New State: SPECTATE
[Project Manager] Current State: SPECTATE
[Project Manager] Received message(s)
[Project Manager] New State: MUTATE
[Software Developer] Current State: SPECTATE
[Software Developer] New State: SPECTATE
[Roadmap Architect] Current State: SPECTATE
[Roadmap Architect] New State: SPECTATE
[Project Manager] Current State: MUTATE
[Project Manager] Internal state mutated


...

Output()

Output()

Output()

Output()

Output()